In [1]:
import pandas as pd

In [3]:
file = pd.read_excel('/home/hrobbin/python/mysql_scripts/JivoChat Report 21-09-02 61305d4165b6d.xlsx', sheet_name=0)
collist = ['x' + str(x) for x in range(13)]
file.columns = collist

In [12]:
start = file.loc[file['x0'] == 'Сводный отчет по сайтам'].index
ends = file.loc[file['x0'] == 'ИТОГО'].index
file2 = pd.DataFrame(file.iloc[start[0] + 3:ends[0] + 1])
file2.rename(columns={'x0': 'site_name', 'x1': 'dial_accepted', 'x2': 'call_accepted', 'x3': 'dial_naccepted', 'x4': 'call_naccepted', 'x5': 'dial_without_reply', 'x6': 'offline_msg'}, inplace=True)
file2 = file2.iloc[:,:-6]
file2

,site_name,dial_accepted,call_accepted,dial_naccepted,call_naccepted,dial_without_reply,offline_msg
7,tyreplus.ru,1,0,0,0,0,0
8,[Yandex] tyreplus.ru,0,0,0,0,0,0
9,michelin.ru,10,0,1,0,1,0
10,VK [Мишлен],0,0,0,0,0,0
11,BFGoodrich.ru,0,0,0,0,0,0
12,[Yandex] BFGoodrich,0,0,0,0,0,0
13,Мишлен,1,0,0,0,0,1
14,savefuel.michelin.ru,0,0,0,0,0,0
15,b2b experience,0,0,0,0,0,0
16,ИТОГО,12,0,1,0,1,1
